# Análisis de ventas de productos digitales simuladas 
## Fase 2. Análisis exploratorio 
Comprender el comportamiento general del negocio y sus dimensiones clave

```mermaid
    graph LR
    A[1 <br> KPIs globales]
    B[2 <br> EDA por dimensiones]
    C[3 <br> Cohorts y recurrencia]
    
    A-->B
    B-->C

    style A fill: #105e6c
    style B fill: #318291
    style C fill: #325b63

```

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv('C:/Users/nat27/Desktop/Desktop/Proyectos/CienciaDatos/digital-sales-analytics/data/clean/digital_products_sales_cleaned.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          8000 non-null   int64  
 1   customer_id       8000 non-null   int64  
 2   product_name      8000 non-null   object 
 3   category          8000 non-null   object 
 4   price_usd         8000 non-null   float64
 5   quantity          8000 non-null   int64  
 6   discount_rate     8000 non-null   float64
 7   gross_amount_usd  8000 non-null   float64
 8   net_revenue_usd   8000 non-null   float64
 9   purchase_dt       8000 non-null   object 
 10  region            8000 non-null   object 
 11  channel           8000 non-null   object 
 12  payment_method    8000 non-null   object 
 13  order_date        8000 non-null   object 
 14  year              8000 non-null   int64  
 15  month             8000 non-null   int64  
 16  day               8000 non-null   int64  


In [13]:
# 1. KPIs globales  
total_net_revenue = data['net_revenue_usd'].sum()
total_gross_revenue = data['gross_amount_usd'].sum()
total_units_sold = data['quantity'].sum()
average_order_value = total_net_revenue / total_orders
maximum_order_value = data['net_revenue_usd'].max()
minimum_order_value = data['net_revenue_usd'].min()
average_discount = data['discount_rate'].mean()
print(f"Total Net Revenue: ${total_net_revenue:,.2f}")
print(f"Total Gross Revenue: ${total_gross_revenue:,.2f}")
print(f"Total Units Sold: {total_units_sold:,}")
print(f"Average Order Value: ${average_order_value:,.2f}")
print(f"Maximum Order Value: ${maximum_order_value:,.2f}")
print(f"Minimum Order Value: ${minimum_order_value:,.2f}")
print(f"Average Discount Rate: {average_discount:.2%}")

Total Net Revenue: $374,647.04
Total Gross Revenue: $379,572.66
Total Units Sold: 9,664
Average Order Value: $46.83
Maximum Order Value: $236.28
Minimum Order Value: $2.55
Average Discount Rate: 1.29%
